In [1]:
import CaboCha
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams["font.family"] = "IPAexGothic"

import pydot_ng as pydot

In [2]:
#自分作成
#最初に読み込んだファイルの状態で、forループをしないと、
#一語一語という解釈になってしまい、「吾」「輩」というような切れ目になってしまう。
with open("neko.txt","r",encoding = "utf-8_sig") as f:
    with open("neko.txt.cabocha","w",encoding = "utf-8_sig") as g:
        c = CaboCha.Parser()
        for i in f:
            g.write(c.parse(i).toString(CaboCha.FORMAT_LATTICE))

In [17]:
#参考サイト1 (neko2.txt.cabochaとして保存)
#参考サイト2,自分とは少し異なるが、Byte数は一緒だから同じデータ？？
#https://qiita.com/sh05_sh05/items/434ab60449f230f95e77
import CaboCha
c = CaboCha.Parser()
with open('neko.txt',encoding = "utf-8_sig") as f:
    text = f.read()
with open('neko2.txt.cabocha', mode='w',encoding = "utf-8_sig") as f:
    for se in  [s + '。' for s in text.split('。')]:
        f.write(c.parse(se ).toString(CaboCha.FORMAT_LATTICE))

In [19]:
#参考サイト2
# https://qiita.com/segavvy/items/2f686cfb0065c8cbe698
import CaboCha

fname = 'neko.txt'
fname_parsed = 'neko3.txt.cabocha'


def parse_neko():
    '''「吾輩は猫である」を係り受け解析
    「吾輩は猫である」(neko.txt)を係り受け解析してneko.txt.cabochaに保存する
    '''
    with open(fname,encoding = "utf-8_sig") as data_file, \
            open(fname_parsed, mode='w',encoding = "utf-8_sig") as out_file:

        cabocha = CaboCha.Parser()
        for line in data_file:
            out_file.write(
                cabocha.parse(line).toString(CaboCha.FORMAT_LATTICE)
            )

parse_neko()

## 第40問

In [1]:
#CaboChaでは、値渡しはやめておく。
#fは一応グローバルな変数としておく。
#イテレータ・ジェネレータは以下のサイトが詳しい
#https://qiita.com/tomotaka_ito/items/35f3eb108f587022fa09

with open("neko.txt.cabocha","r",encoding="utf-8_sig") as f:
    
    class Morph:
        #表層形,基本形,品詞,品詞細分類1の順
        def __init__(self,surface,base,pos,pos1):
            self.surface = surface
            self.base = base
            self.pos = pos
            self.pos1 = pos1
    
        #イテレータ
        def __str__(self):
            return "surface:{}\t base:{}\t pos:{}\t pos1:{}".format(self.surface, self.base, self.pos, self.pos1)

    def Analysis_Morph():
        morphs = []
        flag = 0
        #flagを導入した理由
        #EOSが2回続いた場合、2回目の場所が空リストとなって実行されてしまうため
        
        for i in f:
        
            if i == "EOS\n" and flag > 0:
                flag = 0
                continue
            
            elif i == "EOS\n":
                #一文が終了したことの処理
                flag += 1 
                yield morphs #表示の処理を実行
                morphs = [] #空リストに戻す
            
            else:
                #*で始まる行はスキップ
                if i[0] == "*":
                    flag = 0
                    continue
                    
                else:
                    flag = 0
                    ls = i.split("\t") 
                    ks = ls[1].split(",")
                    #ls[0]=surface, ks[0]=base, ks[1]=pos, ks[6]=pos1
                    morphs.append(Morph(ls[0],ks[6],ks[0],ks[1]))
        
        #raise StopIteration() #yield文がなくなった時用の例外処理(無くても大丈夫？)
        
    k = Analysis_Morph() #リストの作成

    l = int(input("配列番号で何番目？"))
    
    for n,items in enumerate(k):
        if n == l: #要素は0番目スタートなので、3番目は配列で言うと2番目
            for item in items:
                print(item)
            break

配列番号で何番目？2
surface:名前	 base:名前	 pos:名詞	 pos1:一般
surface:は	 base:は	 pos:助詞	 pos1:係助詞
surface:まだ	 base:まだ	 pos:副詞	 pos1:助詞類接続
surface:無い	 base:無い	 pos:形容詞	 pos1:自立
surface:。	 base:。	 pos:記号	 pos1:句点


## 第41問

In [25]:
#CaboChaでは、値渡しはやめておく。
#fは一応グローバルな変数としておく。
#イテレータ・ジェネレータは以下のサイトが詳しい
#https://qiita.com/tomotaka_ito/items/35f3eb108f587022fa09

import re

with open("neko.txt.cabocha","r",encoding="utf-8_sig") as f:
    
    class Morph:
        #表層形,基本形,品詞,品詞細分類1の順
        def __init__(self,surface,base,pos,pos1):
            self.surface = surface
            self.base = base
            self.pos = pos
            self.pos1 = pos1
    
        #イテレータ
        def __str__(self):
            return "surface:{}\t base:{}\t pos:{}\t pos1:{}".format(self.surface, self.base, self.pos, self.pos1)

    class Chunk:
        #形態素（Morphオブジェクト）のリスト,係り先文節インデックス番号,係り元文節インデックス番号のリスト（srcs）
        
        def __init__(self):
            self.morphs = []
            self.dst = -1 #index番号は-1としておく
            self.srcs = []
        
        def __str__(self):
            surface = ""
            for morph in self.morphs:
                surface += morph.surface
            return "words:{}\t dst:{}\t srcs:{}".format(surface,self.dst,self.srcs)
    
    def Analysis_Chunk():
        chunks = dict()
        idx = -1 #keyとなるindex number
        flag = 0
        #flagを導入した理由
        #EOSが2回続いた場合、2回目の場所が空リストとなって実行されてしまうため
        
        for i in f:
        
            if i == "EOS\n" and flag > 0:
                flag = 0
                continue
            
            elif i == "EOS\n":
                #一文が終了したことの処理
                flag += 1
                
                if len(chunks) > 0:
                    data = sorted(chunks.items(),key=lambda x:x[0]) 
                    #このように条件指定することで、配列の0番目がKeyとなりSortしてくれる
                    yield list(zip(*data))[1] #2データ以上ある時は、zipを使う
                    chunks.clear()
                
                else:
                    yield []
                    
            elif i[0] == "*":
                #*で始まる行は「係受け解析結果」
                flag = 0
                #chunkのインデックス番号(idx)と係先のインデックス番号(dst)を取得
                hs = i.split(" ")
                idx = int(hs[1])
                dst = int(hs[2].strip("D"))
                
                #chunkが無ければ生成し、係先のインデックス番号をセット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                    
                #係先のchunkを生成し、係元のインデックス番号を追加しておく
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
                    
            else:
                #この部分は形態素解析結果
                flag = 0
                ls = i.split("\t") 
                ks = ls[1].split(",")
                #ls[0]=surface, ks[0]=base, ks[1]=pos, ks[6]=pos1
                chunks[idx].morphs.append(Morph(ls[0],ks[6],ks[0],ks[1]))
        
        #raise StopIteration() #yield文がなくなった時用の例外処理(無くても大丈夫？)
    
    Analysis_Chunk()
    
    k = Analysis_Chunk()

    l = int(input("配列番号で何番目？"))
    
    for n,items in enumerate(k):
        if n == l:
            for n1,item in enumerate(items):
                print("[{}]{}".format(n1,item))
            break

配列番号で何番目？3
[0]words:　どこで	 dst:1	 srcs:[]
[1]words:生れたか	 dst:4	 srcs:[0]
[2]words:とんと	 dst:4	 srcs:[]
[3]words:見当が	 dst:4	 srcs:[]
[4]words:つかぬ。	 dst:-1	 srcs:[1, 2, 3]


## 第42問

In [51]:
#CaboChaでは、値渡しはやめておく。
#fは一応グローバルな変数としておく。
#イテレータ・ジェネレータは以下のサイトが詳しい
#https://qiita.com/tomotaka_ito/items/35f3eb108f587022fa09

import re

with open("neko.txt.cabocha","r",encoding="utf-8_sig") as f:
    
    class Morph:
        #表層形,基本形,品詞,品詞細分類1の順
        def __init__(self,surface,base,pos,pos1):
            self.surface = surface
            self.base = base
            self.pos = pos
            self.pos1 = pos1
    
        #イテレータ
        def __str__(self):
            return "surface:{}\t base:{}\t pos:{}\t pos1:{}".format(self.surface, self.base, self.pos, self.pos1)

    class Chunk:
        #形態素（Morphオブジェクト）のリスト,係り先文節インデックス番号,係り元文節インデックス番号のリスト（srcs）
        
        def __init__(self):
            self.morphs = []
            self.dst = -1 #index番号は-1としておく
            self.srcs = []
        
        def __str__(self):
            surface = ""
            for morph in self.morphs:
                surface += morph.surface
            return "words:{}\t dst:{}\t srcs:{}".format(surface,self.dst,self.srcs)
        
        def clause_text(self):
            #句読点を除いた表層形を返す
            result = ""
            for morph in self.morphs:
                if morph.pos != "記号":
                    result = result + morph.surface
            return result
            
    
    def Analysis_Chunk():
        chunks = dict()
        idx = -1 #keyとなるindex number
        flag = 0
        #flagを導入した理由
        #EOSが2回続いた場合、2回目の場所が空リストとなって実行されてしまうため
        
        for i in f:
        
            if i == "EOS\n" and flag > 0:
                flag = 0
                continue
            
            elif i == "EOS\n":
                #一文が終了したことの処理
                flag += 1
                
                if len(chunks) > 0:
                    data = sorted(chunks.items(),key=lambda x:x[0]) 
                    #このように条件指定することで、配列の0番目がKeyとなりSortしてくれる
                    yield list(zip(*data))[1] #2データ以上ある時は、zipを使う
                    chunks.clear()
                
                else:
                    yield []
                    
            elif i[0] == "*":
                #*で始まる行は「係受け解析結果」
                flag = 0
                #chunkのインデックス番号(idx)と係先のインデックス番号(dst)を取得
                hs = i.split(" ")
                idx = int(hs[1])
                dst = int(hs[2].strip("D"))
                
                #chunkが無ければ生成し、係先のインデックス番号をセット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                    
                #係先のchunkを生成し、係元のインデックス番号を追加しておく
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
                    
            else:
                #この部分は形態素解析結果
                flag = 0
                ls = i.split("\t") 
                ks = ls[1].split(",")
                #ls[0]=surface, ks[0]=base, ks[1]=pos, ks[6]=pos1
                chunks[idx].morphs.append(Morph(ls[0],ks[6],ks[0],ks[1]))
        
        #raise StopIteration() #yield文がなくなった時用の例外処理(無くても大丈夫？)
    
    Analysis_Chunk()
    
    k = Analysis_Chunk()
    x = []
    y = []
    
    #1文ずつリストを作成
    for items in k:
        for item in items:
            #係先があるものだけ取ってくる
            if item.dst != -1: #初期化のままではない
                src = item.clause_text()
                dst = items[item.dst].clause_text() #一文内のその係先のテキストだけ持ってくる
                if src != "" and dst != "":
                    #print("{}\t{}".format(src,dst))
                    x.append(src)
                    y.append(dst)
    
    with open("neko_result42.txt","w",encoding = "utf-8_sig") as g:
        for i in range(len(x)):
            g.write(x[i] + "\t" + y[i] + "\n")
    
    import pickle
    with open("neko_result42","wb") as h:
        for j in range(len(x)):
            pickle.dump(j,h)
    
            

## 第43問

In [52]:
#CaboChaでは、値渡しはやめておく。
#fは一応グローバルな変数としておく。
#イテレータ・ジェネレータは以下のサイトが詳しい
#https://qiita.com/tomotaka_ito/items/35f3eb108f587022fa09

import re

with open("neko.txt.cabocha","r",encoding="utf-8_sig") as f:
    
    class Morph:
        #表層形,基本形,品詞,品詞細分類1の順
        def __init__(self,surface,base,pos,pos1):
            self.surface = surface
            self.base = base
            self.pos = pos
            self.pos1 = pos1
    
        #イテレータ
        def __str__(self):
            return "surface:{}\t base:{}\t pos:{}\t pos1:{}".format(self.surface, self.base, self.pos, self.pos1)

    class Chunk:
        #形態素（Morphオブジェクト）のリスト,係り先文節インデックス番号,係り元文節インデックス番号のリスト（srcs）
        
        def __init__(self):
            self.morphs = []
            self.dst = -1 #index番号は-1としておく
            self.srcs = []
        
        def __str__(self):
            surface = ""
            for morph in self.morphs:
                surface += morph.surface
            return "words:{}\t dst:{}\t srcs:{}".format(surface,self.dst,self.srcs)
        
        def clause_text(self):
            #句読点を除いた表層形を返す
            result = ""
            for morph in self.morphs:
                if morph.pos != "記号":
                    result = result + morph.surface
            return result
        
        def check_pos(self,pos):
            #名詞を含む文節かどうかを判断
            for morph in self.morphs:
                if morph.pos == pos:
                    return True
                else:
                    return False
            
    
    def Analysis_Chunk():
        chunks = dict()
        idx = -1 #keyとなるindex number
        flag = 0
        #flagを導入した理由
        #EOSが2回続いた場合、2回目の場所が空リストとなって実行されてしまうため
        
        for i in f:
        
            if i == "EOS\n" and flag > 0:
                flag = 0
                continue
            
            elif i == "EOS\n":
                #一文が終了したことの処理
                flag += 1
                
                if len(chunks) > 0:
                    data = sorted(chunks.items(),key=lambda x:x[0]) 
                    #このように条件指定することで、配列の0番目がKeyとなりSortしてくれる
                    yield list(zip(*data))[1] #2データ以上ある時は、zipを使う
                    chunks.clear()
                
                else:
                    yield []
                    
            elif i[0] == "*":
                #*で始まる行は「係受け解析結果」
                flag = 0
                #chunkのインデックス番号(idx)と係先のインデックス番号(dst)を取得
                hs = i.split(" ")
                idx = int(hs[1])
                dst = int(hs[2].strip("D"))
                
                #chunkが無ければ生成し、係先のインデックス番号をセット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                    
                #係先のchunkを生成し、係元のインデックス番号を追加しておく
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
                    
            else:
                #この部分は形態素解析結果
                flag = 0
                ls = i.split("\t") 
                ks = ls[1].split(",")
                #ls[0]=surface, ks[0]=base, ks[1]=pos, ks[6]=pos1
                chunks[idx].morphs.append(Morph(ls[0],ks[6],ks[0],ks[1]))
        
        #raise StopIteration() #yield文がなくなった時用の例外処理(無くても大丈夫？)
    
    Analysis_Chunk()
    
    k = Analysis_Chunk()
    x = []
    y = []
    
    #1文ずつリストを作成
    for items in k:
        for item in items:
            #係先があるものだけ取ってくる
            if item.dst != -1: #初期化のままではない
                if (item.check_pos("名詞") == True) and (items[item.dst].check_pos("動詞") == True):
                    src = item.clause_text()
                    dst = items[item.dst].clause_text() #一文内のその係先のテキストだけ持ってくる
                    if src != "" and dst != "":
                        #print("{}\t{}".format(src,dst))
                        x.append(src)
                        y.append(dst)
    
    with open("neko_result43.txt","w",encoding = "utf-8_sig") as g:
        for i in range(len(x)):
            g.write(x[i] + "\t" + y[i] + "\n")
    
    import pickle
    with open("neko_result43","wb") as h:
        for j in range(len(x)):
            pickle.dump(j,h)
    
            

## 第44問

In [5]:
with open("neko_tmp.txt","w",encoding = "utf-8_sig") as h:
    h.write(input("グラフにする文字列を入力----->"))

グラフにする文字列を入力----->ああ我こそは偉大なる王様であるぞ。


In [7]:
with open("neko_tmp.txt","r",encoding = "utf-8_sig") as f:
    with open("neko_tmp.txt.cabocha","w",encoding = "utf-8_sig") as g:
        c = CaboCha.Parser()
        for i in f:
            g.write(c.parse(i).toString(CaboCha.FORMAT_LATTICE))

In [8]:
#CaboChaでは、値渡しはやめておく。
#fは一応グローバルな変数としておく。
#イテレータ・ジェネレータは以下のサイトが詳しい
#https://qiita.com/tomotaka_ito/items/35f3eb108f587022fa09

#以下は参照サイト
#https://qiita.com/segavvy/items/d1a9a8d87d8dc10a8f15

import re
import pydot_ng as pydot

with open("neko_tmp.txt.cabocha","r",encoding="utf-8_sig") as f:
    
    class Morph:
        #表層形,基本形,品詞,品詞細分類1の順
        def __init__(self,surface,base,pos,pos1):
            self.surface = surface
            self.base = base
            self.pos = pos
            self.pos1 = pos1
    
        #イテレータ
        def __str__(self):
            return "surface:{}\t base:{}\t pos:{}\t pos1:{}".format(self.surface, self.base, self.pos, self.pos1)

    class Chunk:
        #形態素（Morphオブジェクト）のリスト,係り先文節インデックス番号,係り元文節インデックス番号のリスト（srcs）
        
        def __init__(self):
            self.morphs = []
            self.dst = -1 #index番号は-1としておく
            self.srcs = []
        
        def __str__(self):
            surface = ""
            for morph in self.morphs:
                surface += morph.surface
            return "words:{}\t dst:{}\t srcs:{}".format(surface,self.dst,self.srcs)
        
        def clause_text(self):
            #句読点を除いた表層形を返す
            result = ""
            for morph in self.morphs:
                if morph.pos != "記号":
                    result = result + morph.surface
            return result
        
        def check_pos(self,pos):
            #名詞を含む文節かどうかを判断
            for morph in self.morphs:
                if morph.pos == pos:
                    return True
                else:
                    return False
            
    
    def Analysis_Chunk():
        chunks = dict()
        idx = -1 #keyとなるindex number
        flag = 0
        #flagを導入した理由
        #EOSが2回続いた場合、2回目の場所が空リストとなって実行されてしまうため
        
        for i in f:
        
            if i == "EOS\n" and flag > 0:
                flag = 0
                continue
            
            elif i == "EOS\n":
                #一文が終了したことの処理
                flag += 1
                
                if len(chunks) > 0:
                    data = sorted(chunks.items(),key=lambda x:x[0]) 
                    #このように条件指定することで、配列の0番目がKeyとなりSortしてくれる
                    yield list(zip(*data))[1] #2データ以上ある時は、zipを使う
                    chunks.clear()
                
                else:
                    yield []
                    
            elif i[0] == "*":
                #*で始まる行は「係受け解析結果」
                flag = 0
                #chunkのインデックス番号(idx)と係先のインデックス番号(dst)を取得
                hs = i.split(" ")
                idx = int(hs[1])
                dst = int(hs[2].strip("D"))
                
                #chunkが無ければ生成し、係先のインデックス番号をセット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                    
                #係先のchunkを生成し、係元のインデックス番号を追加しておく
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
                    
            else:
                #この部分は形態素解析結果
                flag = 0
                ls = i.split("\t") 
                ks = ls[1].split(",")
                #ls[0]=surface, ks[0]=base, ks[1]=pos, ks[6]=pos1
                chunks[idx].morphs.append(Morph(ls[0],ks[6],ks[0],ks[1]))
        
        #raise StopIteration() #yield文がなくなった時用の例外処理(無くても大丈夫？)
    
    def Making_Tree(tree_list,direct=True):
        
        if direct == True:
            graph = pydot.Dot(graph_type="graph")
        else:
            graph = pydot.Dot(graph_type="digraph")
            
        for tree in tree_list:
            
            #node用のidとlabelの用意
            id1 = str(tree[0][0])
            text1 = str(tree[0][1])
            id2 = str(tree[1][0])
            text2 = str(tree[1][1])
            
            #ノードの追加
            graph.add_node(pydot.Node(id1,label=text1,fontname="IPAexGothic"))
            graph.add_node(pydot.Node(id2,label=text2,fontname="IPAexGothic"))
            
            #エッジの追加
            graph.add_edge(pydot.Edge(id1,id2))
            
        return graph
    
    
    k = Analysis_Chunk()
    
    #1文ずつリストを作成
    for items in k:
        
        trees = []
        
        for i,item in enumerate(items): #2つの値を取ってきたい⇒リスト化
            #係先があるものだけ取ってくる
            if item.dst != -1: #初期化のままではない
                src = item.clause_text()
                dst = items[item.dst].clause_text() #一文内のその係先のテキストだけ持ってくる
                if src != "" and dst != "":
                    #print("{}\t{}".format(src,dst))
                    trees.append(((i,src),(item.dst,dst))) #idはi,item.dst , #label（モノ）はsrc,dst
        
        if len(trees) > 0:
            graph = Making_Tree(trees,direct=True)
            graph.write_png("result44.png")
            


(dot.exe:23180): Pango-WARNING **: couldn't load font "IPAexGothic Not-Rotated 14", falling back to "Sans Not-Rotated 14", expect ugly output.



## 第45問

In [25]:
#CaboChaでは、値渡しはやめておく。
#fは一応グローバルな変数としておく。
#イテレータ・ジェネレータは以下のサイトが詳しい
#https://qiita.com/tomotaka_ito/items/35f3eb108f587022fa09

import re

with open("neko.txt.cabocha","r",encoding="utf-8_sig") as f:
    
    class Morph:
        #表層形,基本形,品詞,品詞細分類1の順
        def __init__(self,surface,base,pos,pos1):
            self.surface = surface
            self.base = base
            self.pos = pos
            self.pos1 = pos1
    
        #イテレータ
        def __str__(self):
            return "surface:{}\t base:{}\t pos:{}\t pos1:{}".format(self.surface, self.base, self.pos, self.pos1)

    class Chunk:
        #形態素（Morphオブジェクト）のリスト,係り先文節インデックス番号,係り元文節インデックス番号のリスト（srcs）
        
        def __init__(self):
            self.morphs = []
            self.dst = -1 #index番号は-1としておく
            self.srcs = []
        
        def __str__(self):
            surface = ""
            for morph in self.morphs:
                surface += morph.surface
            return "words:{}\t dst:{}\t srcs:{}".format(surface,self.dst,self.srcs)
        
        def clause_text(self):
            #句読点を除いた表層形を返す
            result = ""
            for morph in self.morphs:
                if morph.pos != "記号":
                    result = result + morph.surface
            return result
        
        def check_pos(self,pos):
            #名詞を含む文節かどうかを判断
            for morph in self.morphs:
                if morph.pos == pos:
                    return True
                else:
                    return False
        
        #指定した品詞と品詞細分類のリストを返す。
        #品詞細分類の指定が無ければ、品詞のみでも判定
        def get_morphs_by_pos(self,pos,pos1=""):
            
            result = []
            
            if len(pos1) > 0:
                for i in self.morphs:
                    if (i.pos == pos) and (i.pos1 == pos1):
                        result.append(i)
            
            else:
                for i in self.morphs:
                    if i.pos == pos:
                        result.append(i)
            
            return result
        
            
    def Analysis_Chunk():
        chunks = dict()
        idx = -1 #keyとなるindex number
        flag = 0
        #flagを導入した理由
        #EOSが2回続いた場合、2回目の場所が空リストとなって実行されてしまうため
        
        for i in f:
        
            if i == "EOS\n" and flag > 0:
                flag = 0
                continue
            
            elif i == "EOS\n":
                #一文が終了したことの処理
                flag += 1
                
                if len(chunks) > 0:
                    data = sorted(chunks.items(),key=lambda x:x[0]) 
                    #このように条件指定することで、配列の0番目がKeyとなりSortしてくれる
                    yield list(zip(*data))[1] #2データ以上ある時は、zipを使う
                    chunks.clear()
                
                else:
                    yield []
                    
            elif i[0] == "*":
                #*で始まる行は「係受け解析結果」
                flag = 0
                #chunkのインデックス番号(idx)と係先のインデックス番号(dst)を取得
                hs = i.split(" ")
                idx = int(hs[1])
                dst = int(hs[2].strip("D"))
                
                #chunkが無ければ生成し、係先のインデックス番号をセット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                    
                #係先のchunkを生成し、係元のインデックス番号を追加しておく
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
                    
            else:
                #この部分は形態素解析結果
                flag = 0
                ls = i.split("\t") 
                ks = ls[1].split(",")
                #ls[0]=surface, ks[0]=base, ks[1]=pos, ks[6]=pos1
                chunks[idx].morphs.append(Morph(ls[0],ks[6],ks[0],ks[1]))
        
        #raise StopIteration() #yield文がなくなった時用の例外処理(無くても大丈夫？)
    
    Analysis_Chunk()
    
    k = Analysis_Chunk()
    x = []
    y = []
    
    #1文ずつリストを作成
    for items in k:
        for item in items:
            #動詞があるかどうかのチェック
            verbs = item.get_morphs_by_pos("動詞")
            
            if len(verbs) < 1:
                continue
            
            #係元を列挙
            prts = []
            for src in item.srcs:
                #助詞の検索
                prts_in_item = items[src].get_morphs_by_pos("助詞")
                if len(prts_in_item) > 1:
                    #助詞が2つ以上ある場合
                    #助詞よりも格助詞を優先
                    
                    kaku_prts = items[src].get_morphs_by_pos("助詞","格助詞")
            
                    if len(kaku_prts) > 0:
                        prts_in_item = kaku_prts
                
                if len(prts_in_item) > 0:
                    prts.append(prts_in_item[-1])
            
            if len(prts) < 1:
                continue
                
            x.append("{}\t{}\n".format(verbs[0].base," ".join(sorted(prt.surface for prt in prts))))
    
    with open("neko_result45.txt","w",encoding = "utf-8_sig") as g:
        for i in range(len(x)):
            g.write(x[i])
            
    import pickle
    with open("neko_result45","wb") as h:
        for i in range(len(x)):
            pickle.dump(i,h)

## 第46問

In [27]:
#CaboChaでは、値渡しはやめておく。
#fは一応グローバルな変数としておく。
#イテレータ・ジェネレータは以下のサイトが詳しい
#https://qiita.com/tomotaka_ito/items/35f3eb108f587022fa09

import re

with open("neko.txt.cabocha","r",encoding="utf-8_sig") as f:
    
    class Morph:
        #表層形,基本形,品詞,品詞細分類1の順
        def __init__(self,surface,base,pos,pos1):
            self.surface = surface
            self.base = base
            self.pos = pos
            self.pos1 = pos1
    
        #イテレータ
        def __str__(self):
            return "surface:{}\t base:{}\t pos:{}\t pos1:{}".format(self.surface, self.base, self.pos, self.pos1)

    class Chunk:
        #形態素（Morphオブジェクト）のリスト,係り先文節インデックス番号,係り元文節インデックス番号のリスト（srcs）
        
        def __init__(self):
            self.morphs = []
            self.dst = -1 #index番号は-1としておく
            self.srcs = []
        
        def __str__(self):
            surface = ""
            for morph in self.morphs:
                surface += morph.surface
            return "words:{}\t dst:{}\t srcs:{}".format(surface,self.dst,self.srcs)
        
        def clause_text(self):
            #句読点を除いた表層形を返す
            result = ""
            for morph in self.morphs:
                if morph.pos != "記号":
                    result = result + morph.surface
            return result
        
        def check_pos(self,pos):
            #名詞を含む文節かどうかを判断
            for morph in self.morphs:
                if morph.pos == pos:
                    return True
                else:
                    return False
        
        #指定した品詞と品詞細分類のリストを返す。
        #品詞細分類の指定が無ければ、品詞のみでも判定
        def get_morphs_by_pos(self,pos,pos1=""):
            
            result = []
            
            if len(pos1) > 0:
                for i in self.morphs:
                    if (i.pos == pos) and (i.pos1 == pos1):
                        result.append(i)
            
            else:
                for i in self.morphs:
                    if i.pos == pos:
                        result.append(i)
            
            return result
        
        def get_jyosi(self):
            
            #助詞を1つ返す。
            #前問の一部を関数化
            
            prts = self.get_morphs_by_pos("助詞")
            if len(prts) > 1:
                kaku_prts = self.get_morphs_by_pos("助詞","格助詞")
                if len(kaku_prts) > 0:
                    prts = kaku_prts
                    
            if len(prts) > 0:
                return prts[-1].surface
            else:
                return ""

            
    def Analysis_Chunk():
        chunks = dict()
        idx = -1 #keyとなるindex number
        flag = 0
        #flagを導入した理由
        #EOSが2回続いた場合、2回目の場所が空リストとなって実行されてしまうため
        
        for i in f:
        
            if i == "EOS\n" and flag > 0:
                flag = 0
                continue
            
            elif i == "EOS\n":
                #一文が終了したことの処理
                flag += 1
                
                if len(chunks) > 0:
                    data = sorted(chunks.items(),key=lambda x:x[0]) 
                    #このように条件指定することで、配列の0番目がKeyとなりSortしてくれる
                    yield list(zip(*data))[1] #2データ以上ある時は、zipを使う
                    chunks.clear()
                
                else:
                    yield []
                    
            elif i[0] == "*":
                #*で始まる行は「係受け解析結果」
                flag = 0
                #chunkのインデックス番号(idx)と係先のインデックス番号(dst)を取得
                hs = i.split(" ")
                idx = int(hs[1])
                dst = int(hs[2].strip("D"))
                
                #chunkが無ければ生成し、係先のインデックス番号をセット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                    
                #係先のchunkを生成し、係元のインデックス番号を追加しておく
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
                    
            else:
                #この部分は形態素解析結果
                flag = 0
                ls = i.split("\t") 
                ks = ls[1].split(",")
                #ls[0]=surface, ks[0]=base, ks[1]=pos, ks[6]=pos1
                chunks[idx].morphs.append(Morph(ls[0],ks[6],ks[0],ks[1]))
        
        #raise StopIteration() #yield文がなくなった時用の例外処理(無くても大丈夫？)
    
    Analysis_Chunk()
    
    k = Analysis_Chunk()
    x = []
    y = []
    
    #1文ずつリストを作成
    for items in k:
        for item in items:
            #動詞があるかどうかのチェック
            verbs = item.get_morphs_by_pos("動詞")
            
            if len(verbs) < 1:
                continue
            
            #係元に助詞を含む文節を列挙
            items_include_prt = []
            for src in item.srcs:
                if len(items[src].get_jyosi()) > 0:
                    items_include_prt.append(items[src])
                if len(items_include_prt) < 1:
                    continue
                
                #辞書順でソート
                items_include_prt.sort(key=lambda x:x.get_jyosi())
                
            x.append("{}\t{}\t{}\n".format(verbs[0].base, " ".join(sorted(item.get_jyosi() for item in items_include_prt)), " ".join(sorted(item.clause_text() for item in items_include_prt))))
    
    with open("neko_result46.txt","w",encoding = "utf-8_sig") as g:
        for i in range(len(x)):
            g.write(x[i])
            
    import pickle
    with open("neko_result46","wb") as h:
        for i in range(len(x)):
            pickle.dump(i,h)

## 第47問

## 第48問

## 第49問